In [1]:
import pinboard
import datetime
import feedparser
import requests
import datetime

In [22]:
API_KEY = 'API_KEY'

resp = requests.get('https://api.darksky.net/forecast/' + API_KEY + '/' + lat + ',' + long)

lat, long = '40.683', '-73.9708'

low = str(round(resp.json()['daily']['data'][0]['temperatureLow']))
high = str(round(resp.json()['daily']['data'][0]['temperatureHigh']))

if datetime.datetime.fromtimestamp(resp.json()['daily']['data'][0]['time']).strftime("%A") == 'Friday':
    w_low = str(round(min(resp.json()['daily']['data'][1]['temperatureLow'], resp.json()['daily']['data'][2]['temperatureLow'])))
    w_high = str(round(min(resp.json()['daily']['data'][1]['temperatureHigh'], resp.json()['daily']['data'][2]['temperatureHigh'])))
    condition = str(resp.json()['daily']['data'][0]['summary'])
        
    weather = '<p style="text-align: center;">Today -&nbsp;Low: ' + low +  '˚ High: ' + high + '˚<br />' + condition + '<br/>This weekend -&nbsp;Low: ' + w_low +  '˚ High: ' + w_high + '˚</p>'
    
else:
    weather = '<p style="text-align: center;">Today -&nbsp;Low: ' + low +  '˚ High: ' + high + '˚<br />' + condition + '</p>'


pb = pinboard.Pinboard('USER_NAME:API_KEY')

# I publish The Briefly every morning at different times.
# This looks at the RSS feed and grabs the time of the
# most recently published article on thebriefly.com and
# stores the time as pubb.

rss = feedparser.parse('http://thebriefly.com/feed/')
date_time_str = rss['entries'][0]['published'][:-6]
pubb = datetime.datetime.strptime(date_time_str, '%a, %d %b %Y %H:%M:%S')

# Retrieves the 50 most recent pins from my Pinboard account with the
# tag 'nyc' applied and reverses them to reverse chronological order.

listy = pb.posts.recent(count=50, tag='nyc')['posts']
listy.reverse()

# Adds the note from the pins (that's what .extend is) and the url to a list 

briefly = []
for i in range(len(listy)):
    if '?emc=rss&partner=rss' in listy[i].url:
        listy[i].url = listy[i].url.replace('?emc=rss&partner=rss', '')
    if listy[i].time > pubb:
        briefly.append(listy[i].extended + '\n' + listy[i].url)
        
# creates a file

open('the briefly.txt', 'w').close()

# writes to the text file and replaces the two usages of '•'
# to open and close a link.

with open('the briefly.txt', 'w') as f:
    f.write('Pins Saved: ' + str(len(briefly)) + '\n\n')
    f.write(weather)
    for item in briefly:
        if '•' in item:
            f.write(item.split('\n')[0].replace('•', '<a href="' + item.split('\n')[1] + '">', 1).replace('•', '</a>', 1) + '\n\n')
        else:
            f.write('<a href="' + item.split('\n')[1] + '">' + ' '.join(item.split('\n')[0].split('(')[:1]).strip() + '</a>' + ' ' + item.split('\n')[0][len(' '.join(item.split('\n')[0].split('(')[:1])):] + '\n\n')